# Lecture 13- Spring 2025
Villas-Boas

This notebook does the following:

Measuring amenities, hedonic model

Standardizing

Functional forms

Selection x var, models, adjusted R squared

Study Ch 6.1 + 6.3

Today starts the material after the Midterm

---------
Midterm review topics posted on bcourses, with tables for midterm
Practice Midterm posted, with solutions and formula sheet posted

We will give you tables and formula sheet to use for the midterm. 
                                                
You do not bring your sheets
    
Please bring a simple non scientific calculator


In [ ]:
# Load the 'pacman' package
install.packages("pacman")
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)

pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse)
pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse,psych,car)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots

In [ ]:
#set scientific display off, thank you Roy
options(scipen=999)

In [ ]:
#read in a Stata dataset
my_data <- read_dta("Lecture13HPRICE2.dta")
head(my_data)


 
 
 Source:  D. Harrison and D.L. Rubinfeld (1978), "Hedonic Housing Prices and the Demand for Clean Air," 
Journal of Environmental Economics and Management 5, 81-102. (data Lecture13Hprice2.dta in bcourses)
Unit of analysis census tract in the Boston area – Most data 1970 U.S. Census.  
The data below were obtained by merging/ matching average house prices and characteristics by census tract 
(1 to 6 ) with crime (census) levels  and pollution (variable 7) levels from another source.
1. price           median housing price, $
2. crime           crimes committed per capita
3. ppoverty         % of people in poverty'
4. rooms           avg number of rooms per house
5. dist            weighted dist. to 5 employ centers
6. stratio         average student-teacher ratio
7. nox             nitrous oxide, parts per 100 million.  (EPA standard 5.3)


In [ ]:
#***summary stats of the data : price nox crime dist rooms ppoverty stratio

#summary stats of data 
#one way describes all data:
describe(my_data)

In [ ]:
#to describe only a subset
data2<-cbind(my_data$price,my_data$nox,my_data$crime,my_data$dist,my_data$rooms,my_data$ppoverty,my_data$stratio)

##Renaming first four columns columns
colnames(data2) <- c("price", "nox", "crime", "dist", "rooms", "ppoverty", "stratio")

describe(data2)

In [ ]:
#box plot of NOX
boxplot(my_data$nox, main="Nox" ) 
# box plot for 'nox'

#/*NOX: the variable in measured in parts per 100 mill (pp100m)  nitrogen dioxide
#The   EPA official annual standard is 5.3 ppm
#https://www3.epa.gov/ttn/naaqs/standards/nox/s_nox_history.html 
#*/
#  /*REVIEW for MIDTERM: What do you see in terms of the data standard deviation

#and Max Min of annual NOX in US census tracts?  
#  Variable | Obs        Mean    Std. Dev.       Min        Max
#nox |        506    5.549783    1.158395       3.85       8.71
#What is the average NOX among the data census tracts? What is standard error of the average?


#  We know that average 5.549783   
# and std dev of the data in sample is 1.158395

#Answer= std dev of average is = 1.158395/square_root(506)
#*/
  

In [ ]:
reg13 <- lm(price~nox+crime+dist+rooms+ppoverty+stratio, my_data)
summary(reg13)

What do you see in the output of regression above

Holding all other regressors constant will mean ceteris paribus in this notebook.

One particulate increase in nox ceteris paribus is correlated with a predicted price drop by 1757 dollars

One more crime per capita ceteris paribus is correlated with predicted housing price drops by 80 dollars

Do, for yourself, a review for midterm, t test? significance ? pvalues?

# New : 

But how do we compare the importance of these two factors as being correlated with the outcome, when those factors have different means and ranges?

Crime 0 to 89 average 3.6  

Nox average 5.5 pp 100 mill and from 3.8 to 8.71?

Solution, compare them after standardizing the coefficients. And the one with the largest standardized coefficient has the biggest correlation with the outcome

In [ ]:
#New-------------------------
#But how do we compare the importance of the correlations of these two factors  that have different means and ranges?

#  Crime 0 to 89 average 3.6  

#nox average 5.5 pp 100 mill and from 3.8 to 8.71?

#standardize the coefficients then, to do so


#lets write a function
#coefficients:
b <- reg13$coef
X<-cbind(1,my_data$nox,my_data$crime,my_data$dist,my_data$rooms,my_data$ppoverty,my_data$stratio)
sx1<-sd(X[,1])
sx2<-sd(X[,2])
sx3<-sd(X[,3])
sx4<-sd(X[,4])
sx5<-sd(X[,5])
sx6<-sd(X[,6])
sx7<-sd(X[,7])
sx<-cbind(sx1,sx2,sx3,sx4,sx5,sx6,sx7)
sy<-sd(my_data$price)
beta <- b * sx/sy
#pring standardized betas:
beta

From the above results we see that

One std dev increase in Nox ceteris paribus is correlated with a   price drop by 0.22 standard dev

One std dev increase in crime ceteris paribus is correlated with a  housing price drop by 0.075 std dev

This is how we compare the importance of the correlations of these two factors  that have different means and ranges, 
using Z scores and interpreting the standardized betas

# New:

How do we choose between two models with the same y variable but different X's on the right?

In this case, which model is preferred?

Model 1 $log(price)_i=\beta_1+\beta_2 log(distance)_i+\epsilon_{1i}$

or

Model 2 $log(price)_i=\alpha_1+\alpha_2 distance_i+\alpha_3 distance^2_i+\epsilon_{2i}$

where the second model has distance and the square of distance as regressors, whereas the first model has log 
distance as a regressor. Both have log(price) as the dependent variable.

How do we choose between two models with the same y variable but different X's on the right?




In [ ]:


#logs specification? log of distance

lprice<-log(my_data$price)
ldist<-log(my_data$dist)
lnox<-log(my_data$nox)

reg13log<-lm(lprice~lnox+rooms+ppoverty+ldist,my_data)
summary(reg13log)



In [ ]:

#regress log price on distance and distance squared specification 
my_data$dist2<-my_data$dist*my_data$dist
reg13sq<-lm(lprice~nox+crime+dist+dist2+rooms+ppoverty+stratio, my_data)
summary(reg13sq)



What do you see? 
Which MODEL REGRESSION do we choose in this case?
 
Adj R squared with ldist  0.68

Adj R squared with dist and dist2    0.7565

So you would choose the one with dist and dist 2 
instead of the one with log distance, because it has the higher adjusted R squared

Take away

if same y and different X's then use adjusted R2 like in this lecture

how do we select between a regression of price on nox or a regression of log price on nox?
different method for model selection, in a future lecture. 



The end